https://vgpena.github.io/classifying-tweets-with-keras-and-tensorflow/

En el anterior enlace, tenéis un ejemplo sobre cómo, a partir de tweets con un label específico (un sentimiento, positivo o negativo): 

1. Genera un conjunto de entrenamiento. El conjunto de entrenamiento es formado a partir de tweets completos pasados a un array con un tamaño específico.
2. Ese array (X_train de tamaño N) tiene un label que representa el sentimiento (y_train)
3. Como todas las frases tienen un tamaño N, la entrada de la red neuronal será de tamaño N y la salida de la red será de tamaño 2 usando activación softmax(porque hay dos clases).

Se pide: 

- Realizar un clasificador de reviews para el dataset de IMDB de la carpeta data_exercise/

**Cuando usa la importación "keras.x", reemplázalo por "tensorflow.keras.x"**

In [14]:
#importamos librerias

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from tensorflow import keras
import tensorflow.keras.preprocessing.text as kpt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation

In [15]:
training = pd.read_csv("data_exercise/IMDB Dataset.csv")



In [16]:
training.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [17]:
X_train = training["review"]
y_train = training["sentiment"]

In [18]:
y_train = y_train.apply(lambda x: 1 if x == "positive" else 0)

In [19]:
y_train = np.asarray(y_train)

In [20]:
# only work with the 3000 most popular words found in our dataset
max_words = 3000

In [21]:
# create a new Tokenizer
tokenizer = Tokenizer(num_words=max_words)
# feed our tweets to the Tokenizer
tokenizer.fit_on_texts(train_x)

In [22]:
# Tokenizers come with a convenient list of words and IDs
dictionary = tokenizer.word_index
# Let's save this out so we can use it later
with open('dictionary.json', 'w') as dictionary_file:
    json.dump(dictionary, dictionary_file)

In [23]:
def convert_text_to_index_array(text):
    # one really important thing that `text_to_word_sequence` does
    # is make all texts the same length -- in this case, the length
    # of the longest text in the set.
    return [dictionary[word] for word in kpt.text_to_word_sequence(text)]

In [24]:
allWordIndices = []
# for each tweet, change each token to its ID in the Tokenizer's word_index
for text in train_x:
    wordIndices = convert_text_to_index_array(text)
    allWordIndices.append(wordIndices)

In [25]:
# now we have a list of all tweets converted to index arrays.
# cast as an array for future usage.
allWordIndices = np.asarray(allWordIndices)

In [26]:
# create one-hot matrices out of the indexed tweets
train_x = tokenizer.sequences_to_matrix(allWordIndices, mode='binary')

In [29]:
# treat the labels as categories
train_y = keras.utils.to_categorical(y_train, 2)

In [30]:
model = Sequential()
model.add(Dense(512, input_shape=(max_words,), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

In [31]:
model.compile(loss='categorical_crossentropy',
  optimizer='adam',
  metrics= ["CategoricalAccuracy"])

In [32]:
hist_model = model.fit(train_x, train_y,
  batch_size=32,
  epochs=5,
  verbose=1,
  validation_split=0.1,
  shuffle=True)

Epoch 1/5
1/1 [==============================] - 1s 615ms/step - loss: 0.3498 - categorical_accuracy: 1.0000 - val_loss: 1.3156 - val_categorical_accuracy: 0.0000e+00
Epoch 2/5
1/1 [==============================] - 0s 24ms/step - loss: 1.5967 - categorical_accuracy: 0.0000e+00 - val_loss: 1.2051 - val_categorical_accuracy: 0.0000e+00
Epoch 3/5
1/1 [==============================] - 0s 24ms/step - loss: 0.9582 - categorical_accuracy: 0.0000e+00 - val_loss: 1.0924 - val_categorical_accuracy: 0.0000e+00
Epoch 4/5
1/1 [==============================] - 0s 32ms/step - loss: 0.5486 - categorical_accuracy: 1.0000 - val_loss: 0.9812 - val_categorical_accuracy: 0.0000e+00
Epoch 5/5
1/1 [==============================] - 0s 25ms/step - loss: 0.2347 - categorical_accuracy: 1.0000 - val_loss: 0.8775 - val_categorical_accuracy: 0.0000e+00


In [33]:
model_json = model.to_json()
with open('model.json', 'w') as json_file:
    json_file.write(model_json)

model.save_weights('model_imdb.h5')